### How to run my model pipeline

In [1]:
# load data
import pandas as pd
import numpy as np
import copy
import sklearn
import imblearn
import joblib

# CHANGE FILE PATH
explore=pd.read_excel(r'C:\Users\darre\OneDrive\Desktop\models (ntu datathon)\CTG.xls',sheet_name=1,header=1)
explore = explore.loc[:, ~explore.columns.str.contains('Unnamed:')]
complete_features=['b','e','LB','AC','FM','UC','DL','DS','DP','ASTV','MSTV','ALTV','MLTV','Width','Min','Max','Nmax','Nzeros','Mode','Mean','Median','Variance','Tendency','CLASS','NSP']

explore=explore[complete_features]
df=explore.copy()

#remove Y variables to stimulate test data
df=df.drop(columns=['NSP'])

# ----------external fuctions (ARE REQUIRED) TO LOAD MODEL------------------
 
def add_features(df):
    df = df.copy()
    df['heart_rate_range'] = df['Max'] - df['Min']
    df['ASTV_MSTV_ratio'] = df['ASTV'] / (df['MSTV'] + 1e-8)
    df['ALTV_MLTV_ratio'] = df['ALTV'] / (df['MLTV'] + 1e-8)
    df['Variability_Score'] = (df['ASTV'] + df['ALTV']) / 2
    df['AC_UC_interaction'] = df['AC'] * df['UC']
    return df

def outlier_removal(X, threshold=7):
    new_X = copy.deepcopy(X)
    new_X_scaled = sklearn.preprocessing.scale(X)
    new_X[abs(new_X_scaled) > threshold] = np.nan

    return new_X



def labels_to_results(df, label_col='predicted_label'):
    label_map = {1: 'Normal', 2: 'Suspect', 3: 'Pathologic'}
    df['results'] = df[label_col].map(label_map)
    return df

# ------------RUN MODEL-------------------

# CHANGE FILE PATH
final_model_pipeline = joblib.load(r'C:\Users\darre\OneDrive\Desktop\models (ntu datathon)\best_model_pipeline')

df['pred']=final_model_pipeline.predict(df)
df=labels_to_results(df,label_col='pred')
# pred are the [1,2,3]
#results are [Normal=1; Suspect=2; Pathologic=3]
df


,b,e,LB,AC,FM,UC,DL,DS,DP,ASTV,...,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,CLASS,pred,results
0,240.0,357.0,120.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,...,2.0,0.0,120.0,137.0,121.0,73.0,1.0,9.0,2.0,Suspect
1,5.0,632.0,132.0,4.0,0.0,4.0,2.0,0.0,0.0,17.0,...,6.0,1.0,141.0,136.0,140.0,12.0,0.0,6.0,1.0,Normal
2,177.0,779.0,133.0,2.0,0.0,5.0,2.0,0.0,0.0,16.0,...,5.0,1.0,141.0,135.0,138.0,13.0,0.0,6.0,1.0,Normal
3,411.0,1192.0,134.0,2.0,0.0,6.0,2.0,0.0,0.0,16.0,...,11.0,0.0,137.0,134.0,137.0,13.0,1.0,6.0,1.0,Normal
4,533.0,1147.0,132.0,4.0,0.0,5.0,0.0,0.0,0.0,16.0,...,9.0,0.0,137.0,136.0,138.0,11.0,1.0,2.0,1.0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2124,1576.0,3049.0,140.0,1.0,0.0,9.0,0.0,0.0,0.0,78.0,...,6.0,0.0,152.0,147.0,151.0,4.0,1.0,5.0,2.0,Suspect
2125,2796.0,3415.0,142.0,1.0,1.0,5.0,0.0,0.0,0.0,74.0,...,2.0,1.0,145.0,143.0,145.0,1.0,0.0,1.0,1.0,Normal
2126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Normal
2127,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Normal
